In [1]:
import pandas as pd
import numpy as np

/home/arpit/workspace/arpit/experimental/paper-folds/venv/lib/python3.12/site-packages/numpy/_core/getlimits.py:551: UserWarning: Signature b'\x00\xd0\xcc\xcc\xcc\xcc\xcc\xcc\xfb\xbf\x00\x00\x00\x00\x00\x00' for <class 'numpy.longdouble'> does not match any known type: falling back to type probe function.
This warnings indicates broken support for the dtype!
  machar = _get_machar(dtype)


In [2]:
def calculate_paper_folding(n_folds):
    """
    Calculate paper folding metrics:
    1. Height of fold after n folds (converting from cm to m)
    2. Initial paper dimensions needed to get 1cm after n folds
    
    Args:
        n_folds (int): Number of folds to calculate
        
    Returns:
        pandas.DataFrame: Table with folding calculations
    """
    # Initialize lists to store calculations
    fold_numbers = list(range(1, n_folds + 1))
    
    # Calculate height of fold (2^n * initial thickness)
    # Assuming paper thickness is 0.1mm = 0.01cm
    initial_thickness_cm = 0.01
    heights_m = [(2**n * initial_thickness_cm) / 100 for n in fold_numbers]
    
    # Create DataFrame
    df = pd.DataFrame({
        'Number_of_Folds': fold_numbers,
        'Height_of_Fold_meters': heights_m,
    })
    
    # Round the values for better readability
    df['Height_of_Fold_meters'] = df['Height_of_Fold_meters'].round(6)    
    return df

In [3]:
calculate_paper_folding(50)

,Number_of_Folds,Height_of_Fold_meters
0,1,2.000000e-04
1,2,4.000000e-04
2,3,8.000000e-04
3,4,1.600000e-03
4,5,3.200000e-03
5,6,6.400000e-03
6,7,1.280000e-02
7,8,2.560000e-02
8,9,5.120000e-02
9,10,1.024000e-01


In [4]:
def calculate_initial_dimensions(n_folds):
    """
    Calculate initial paper dimensions needed to get 1cm x 1cm after n folds.
    Each fold halves one dimension.
    
    Args:
        n_folds (int): Number of folds to calculate
        
    Returns:
        pandas.DataFrame: Table with folding dimensions
    """
    # Initialize lists for calculations
    fold_numbers = list(range(1, n_folds + 1))
    initial_lengths = []
    initial_widths = []
    fold_sequences = []
    
    for n in fold_numbers:
        # For each number of folds, we need to distribute them between length and width
        # We'll try all possible combinations of horizontal and vertical folds
        best_length = float('inf')
        best_width = float('inf')
        best_sequence = ''
        
        # Try different combinations of horizontal (H) and vertical (V) folds
        for num_horizontal in range(n + 1):
            num_vertical = n - num_horizontal
            
            # Calculate dimensions
            # Each fold halves the respective dimension
            length = 1 * (2 ** num_horizontal)  # 1cm is target size
            width = 1 * (2 ** num_vertical)     # 1cm is target size
            
            # Keep track of the smallest square paper needed
            # We want the paper to be square, so we take the max dimension
            max_dim = max(length, width)
            if max_dim < max(best_length, best_width):
                best_length = length
                best_width = width
                best_sequence = 'H' * num_horizontal + 'V' * num_vertical
        
        initial_lengths.append(best_length)
        initial_widths.append(best_width)
        fold_sequences.append(best_sequence)
    
    # Create DataFrame
    df = pd.DataFrame({
        'Number_of_Folds': fold_numbers,
        'Initial_Length_cm': initial_lengths,
        'Initial_Width_cm': initial_widths,
        'Minimum_Square_Size_cm': [max(l, w) for l, w in zip(initial_lengths, initial_widths)],
        'Fold_Sequence': fold_sequences,
        'Final_Size': ['1cm x 1cm'] * len(fold_numbers)
    })
    
    # Round numeric values for better readability
    numeric_columns = ['Initial_Length_cm', 'Initial_Width_cm', 'Minimum_Square_Size_cm']
    df[numeric_columns] = df[numeric_columns].round(2)
    
    return df

In [5]:
calculate_initial_dimensions(50)

,Number_of_Folds,Initial_Length_cm,Initial_Width_cm,Minimum_Square_Size_cm,Fold_Sequence,Final_Size
0,1,1,2,2,V,1cm x 1cm
1,2,2,2,2,HV,1cm x 1cm
2,3,2,4,4,HVV,1cm x 1cm
3,4,4,4,4,HHVV,1cm x 1cm
4,5,4,8,8,HHVVV,1cm x 1cm
5,6,8,8,8,HHHVVV,1cm x 1cm
6,7,8,16,16,HHHVVVV,1cm x 1cm
7,8,16,16,16,HHHHVVVV,1cm x 1cm
8,9,16,32,32,HHHHVVVVV,1cm x 1cm
9,10,32,32,32,HHHHHVVVVV,1cm x 1cm
